我的csv文件里的'note'列里面有需要进行截断的文本，['CHIEF COMPLAINT',‘PAST SURGICAL HISTORY’,'REVIEW OF SYSTEMS','EMERGENCY DEPARTMENT COURSE','GYNECOLOGIC HISTORY','PAST MEDICAL HISTORY','HISTORY of PRESENT ILLNESS','FAMILY HISTORY/SOCIAL HISTORY']，当这些节标题出现的时候我需要对note里面的文本进行截断，以这些节标题作为key，以这些节标题后对应的文本为values,保存到字典中，最后以'encounter_id'列里对应的id存到json中

In [29]:
import pandas as pd
import json

# 读取CSV文件
data = pd.read_csv('./MEDIQA-Chat-Training-ValidationSets-Feb-10-2023/TaskB/TaskB-TrainingSet.csv')

# 需要截断的节标题
section_headers = ['CHIEF COMPLAINT','PAST SURGICAL HISTORY','REVIEW OF SYSTEMS','RESULTS','ASSESSMENT AND PLAN','EMERGENCY DEPARTMENT COURSE','GYNECOLOGIC HISTORY','PAST MEDICAL HISTORY','HISTORY OF PRESENT ILLNESS','FAMILY HISTORY/SOCIAL HISTORY','PHYSICAL EXAMINATION','VITALS REVIEWED']

# 用来组成输出json的id列
id_col = 'encounter_id'

def split_text(text, keywords):
    keyword_locations = {}
    for keyword in keywords:
        index = text.find(keyword)
        if index != -1:
            keyword_locations[keyword]=index
        else:
            keyword_locations[keyword]=len(text)
    
    keyword_locations = sorted(keyword_locations.items(), key=lambda d:d[1])
    result = []
    for i in range(len(keyword_locations)):
        start = keyword_locations[i][1] + len(keyword_locations[i][0])
        if i == len(keyword_locations)-1:
            end = len(text)+1 
        else:
            end = keyword_locations[i+1][1]
        if i == 0 and start != 0:
            start = len(keyword_locations[i][0])+1
           
        result.append(text[start:end].strip())
        
    return keyword_locations,result


# 初始化输出json对象
output = []

# 遍历每个encounter_id
for id_value in data[id_col].unique():
    # 创建该encounter_id对应的输出字典
    encounter_dict = {}
    
    # 获取该encounter_id对应的行
    encounter_rows = data[data[id_col] == id_value]
    
    # 遍历该encounter_id对应的所有行
    for index, row in encounter_rows.iterrows():
        # section_header = row['section_header']
        note = row['note']
        section_header=[]
        for sec in section_headers:
            if sec in note:
                # print(sec)
                section_header.append(sec)
        # 判断该行是否包含需要截断的节标题
        # if section_header in section_headers:
        #     # 截断文本
        #     note = note.split(section_header, 1)[-1]
            
        #     # 将截断后的文本以对应的节标题为key，添加到输出字典中
        #     if section_header in encounter_dict:
        #         encounter_dict[section_header] += note
        #     else:
        #         encounter_dict[section_header] = note
        section_dict,res = split_text(keywords= section_header,text=note)
        # print(res)
        for i,sec in enumerate(section_dict):
            # print(i)
            encounter_dict[section_dict[i][0]]= res[i]
    
    # 将该encounter_id和对应的字典添加到输出json对象中
    output.append({id_col: id_value, 'sections': encounter_dict})

# 将输出json对象保存到文件中
with open('output.json', 'w') as f:
    json.dump(output, f)